In [1]:
# Python version > 3

# Model training: data encoding

# Importing modules

# Modules needed for the first part of this notebook:
# !pip install xlrd
# !pip install pandas
# !pip install pickle

import numpy as np
import pandas as pd
import pickle
import random
import re

from io import StringIO
from pandas import Series, DataFrame
from collections import Counter

In [2]:
# Loading and reordering data
df_train = pd.read_excel('log_data_evil.xlsx', index_col=0)
df_train.sort_index(ascending=True, inplace=True)

# Droping columns that will not be used
df_train.drop(['http_user_agent'], inplace=True, axis=1)

In [3]:
# Filtering and formatting the data with regex
df_train.request_url.replace({r'^/$': '<BASE_URL>'}, regex=True, inplace=True)
df_train.request_url.replace({r'\%+[\%0-9A-Za-z]*': ' <PERCENT_URL> '},regex=True, inplace=True)
df_train.request_url.replace({r'\w+\d+\w+': ' '},regex=True, inplace=True)
df_train.request_url.replace({r'\d{2,}': ' '},regex=True, inplace=True)
df_train.request_url.replace({r'\[\d*\]': ' '},regex=True, inplace=True)
df_train.request_url.replace({'/': ' ', ':': ' ', '\.': ' ', '\?': ' ', '=': ' ', '\|': ' ', '&': ' '},regex=True, inplace=True)

In [4]:
# Dropping extra spaces in the url strings
list_aux = [' '.join(word.split()).lower() for word in df_train['request_url'].tolist()]

In [5]:
# Creating a vocabulary for the neuronal network based on the url DataFrame column

# Joinning all the url's in a String and separating all the individual words in an Array
as_one = ' '.join(list_aux)
words = as_one.split()

# Dropping duplicate words
counts = Counter(words)
vocab = sorted(counts, key=counts.get)

# Creating the vocabulary dictionary and predefining some reserved words
word_index = {word: (index + 9) for (index, word) in enumerate(vocab)}
word_index['<PAD>'] = 0
word_index["<URL_START>"] = 1
word_index['<GET>'] = 2
word_index['<POST>'] = 3
word_index['<TEMPORARY_REDIRECT>'] = 4
word_index['<BAD_REQUEST>'] = 5
word_index['<NOT_FOUND>'] = 6
word_index['<OK>'] = 7
word_index['<MOVED_PERMANTLY>'] = 8

# Creating the reverse vocabulary
reverse_word_index = {value: key for (key, value) in word_index.items()}

In [6]:
# Defining functions for handling (encoding and decoding) the urls based on the vocabulary

# Encode a String and return an Array of integers
def encode(text):
    text_ls = []
    for i in text.split():
        try:
            text_ls.append(word_index[i])
        except KeyError:
            print('Unhandled word \'' + i + '\'')
    return text_ls
# Decode an Array of integers and return a String
def decode(ls):
    return ' '.join([reverse_word_index.get(i, '?') for i in ls])

# Encode a single word and return its vocabulary-indexed integer
def encode_single(text):
    return word_index.get(text)
# Decode a single integer and return its (reversed) vocabulary-indexed String
def decode_single(index):
    return reverse_word_index.get(index)

# Helper function: append a '<PAD>' encoded String before a url
def insert_start(ls):
    ls.insert(0, 1)
    return ls

In [7]:
# Saving both vocabularys on disk
with open('abd_variables.pkl', 'wb') as f:
    pickle.dump([word_index, reverse_word_index], f)

In [8]:
# To lower case, then enconding and inserting a '<PAD>' encoded String to every url in the DataFrame
df_train['request_url'] = df_train['request_url'].str.lower()
df_train['request_url'] = df_train['request_url'].apply(encode)
df_train['request_url'] = df_train['request_url'].apply(insert_start)

In [9]:
# Encoding every request in the DataFrame
df_train['request_method'].replace({'GET': 2, 'POST': 3}, inplace=True)

In [10]:
# From status code to String, then encoding every status code in the DataFrame
df_train['status'].replace({307: '<TEMPORARY_REDIRECT>', 400: '<BAD_REQUEST>', 404: '<NOT_FOUND>', 200: '<OK>', 301: '<MOVED_PERMANTLY>'}, inplace=True)
df_train['status'] = df_train['status'].apply(encode_single)

In [11]:
# Separating and dropping the training labels from the DataFrame
total_labels = df_train['is_evil'].values
df_train.drop('is_evil', axis=1, inplace=True)

In [12]:
# Getting the length of the the largest url in the DataFrame
max_length = 0
for i in df_train.values:
    if len(i[2]) > max_length:
        max_length = len(i[2])

print('the largest url have', max_length, 'words')

the largest url have 11 words


In [13]:
# Model training: model definition and training

# Importing modules

# Modules needed for the first part of this notebook:
# !pip install tensorflow

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

print('Tensorflow version:', tf.__version__)

# Printing available computing devices 
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print('Available computing devices:', get_available_devices())

Tensorflow version: 1.14.0
Available computing devices: ['/device:CPU:0', '/device:XLA_CPU:0']


In [14]:
# Normalizing the url length
aux_request_data = keras.preprocessing.sequence.pad_sequences(df_train['request_url'].values,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=max_length)
df_train['request_url'] = Series(aux_request_data.tolist())

In [15]:
# Creating 'max_length' new columns in the DataFrame
for i in range(max_length):
    df_train['request_url' + str(i)] = 0

# Moving the url encoded data from its column to the new columns, one encoded String per column
for i in df_train.itertuples():
    for j in range(max_length):
        df_train.at[i[0], 'request_url' + str(j)] = i[3][j]

# Dropping the request url column in the DataFrame
df_train.drop(['request_url'], axis=1, inplace=True)

# Declare the total data from the values of the DataFrame
total_data = df_train.values

In [16]:
# View the dimensions of the total data and labels variables
print(total_data.shape, total_labels.shape)

# Defining the vocabulary size
vocab_size = len(word_index.keys())
print(vocab_size)

(7716, 13) (7716,)
727


In [17]:
# Defining the neuronal network model
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

# Printing a summary of the model
model.summary()

W0718 11:30:17.935120 140105079375680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 11:30:17.958660 140105079375680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          11632     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 11,921
Trainable params: 11,921
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

W0718 11:30:18.027380 140105079375680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
# Declaring 75% index of the total data
half_length_data = int(len(total_data) * 0.75)

# Separing the data: train data and labels (first 75% of total)
train_data = total_data[:half_length_data]
train_labels = total_labels[:half_length_data]

# Separing the data: test data and labels (second 25% of total)
test_data = total_data[half_length_data:]
test_labels = total_labels[half_length_data:]

In [20]:
# Viewing the lengths of the train and test data
print('Train data lengths:', len(train_data), len(train_labels))
print('Test data lengths:', len(test_data), len(test_labels))

Train data lengths: 5787 5787
Test data lengths: 1929 1929


In [21]:
# Declaring 40% index of the train data
train_length = int(len(train_data) * 0.4)

# Separing the train data: validation data (first 40% of the train data)
x_val = train_data[:train_length]
# Separing the train data: train data to fit the model (second 60% of the train data)
partial_x_train = train_data[train_length:]

# Separing the train labels: validation labels (first 40% of the train labels)
y_val = train_labels[:train_length]
# Separing the train labels: labels to fit the model (second 60% of the train labels)
partial_y_train = train_labels[train_length:]

In [22]:
# Viewing the lengths of the separed train data
print('Fit data and label lengths:', len(partial_x_train), len(partial_y_train))
print('Validate data and label lengths:', len(x_val), len(y_val))

Fit data and label lengths: 3473 3473
Validate data and label lengths: 2314 2314


In [23]:
# Training the model
history = model.fit(partial_x_train,
                    partial_y_train,
                    # Number of epochs to train the model
                    # One epoch: one forward pass and one backward pass of all the training examples
                    # One epoch: one pass over the entire dataset
                    epochs=len(partial_x_train)+len(x_val),
                    # Number of samples per gradient update
                    # 32 is the default size in the Keras framework
                    # The higher the batch size, the more memory space needed
                    # The smaller the batch size (until the length of the training data), the less accurate the estimate of the gradient will be
                    batch_size=len(partial_x_train),
                    validation_data=(x_val, y_val),
                    # Printing the process in progress bar mode
                    verbose=1)

Train on 3473 samples, validate on 2314 samples
Epoch 1/5787
3473/3473 [==============================] - 0s 32us/sample - loss: 0.6939 - acc: 0.1886 - val_loss: 0.6915 - val_acc: 0.8825
Epoch 2/5787
3473/3473 [==============================] - 0s 3us/sample - loss: 0.6915 - acc: 0.9047 - val_loss: 0.6893 - val_acc: 0.8898
Epoch 3/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.6892 - acc: 0.9096 - val_loss: 0.6871 - val_acc: 0.8898
Epoch 4/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.6867 - acc: 0.9096 - val_loss: 0.6849 - val_acc: 0.8898
Epoch 5/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.6843 - acc: 0.9096 - val_loss: 0.6826 - val_acc: 0.8898
Epoch 6/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.6819 - acc: 0.9096 - val_loss: 0.6803 - val_acc: 0.8898
Epoch 7/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.6795 - acc: 0.9096 - val_loss: 0.6780 - v

Epoch 60/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.5205 - acc: 0.9096 - val_loss: 0.5259 - val_acc: 0.8898
Epoch 61/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.5164 - acc: 0.9096 - val_loss: 0.5220 - val_acc: 0.8898
Epoch 62/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.5123 - acc: 0.9096 - val_loss: 0.5181 - val_acc: 0.8898
Epoch 63/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.5081 - acc: 0.9096 - val_loss: 0.5141 - val_acc: 0.8898
Epoch 64/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.5039 - acc: 0.9096 - val_loss: 0.5102 - val_acc: 0.8898
Epoch 65/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.4997 - acc: 0.9096 - val_loss: 0.5062 - val_acc: 0.8898
Epoch 66/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.4955 - acc: 0.9096 - val_loss: 0.5022 - val_acc: 0.8898
Epoch 67/5787
3473/3473 [==

Epoch 119/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2988 - acc: 0.9096 - val_loss: 0.3205 - val_acc: 0.8898
Epoch 120/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2964 - acc: 0.9096 - val_loss: 0.3184 - val_acc: 0.8898
Epoch 121/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2942 - acc: 0.9096 - val_loss: 0.3164 - val_acc: 0.8898
Epoch 122/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2920 - acc: 0.9096 - val_loss: 0.3144 - val_acc: 0.8898
Epoch 123/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2899 - acc: 0.9096 - val_loss: 0.3125 - val_acc: 0.8898
Epoch 124/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2879 - acc: 0.9096 - val_loss: 0.3107 - val_acc: 0.8898
Epoch 125/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2859 - acc: 0.9096 - val_loss: 0.3089 - val_acc: 0.8898
Epoch 126/5787
3473/

Epoch 178/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2289 - acc: 0.9096 - val_loss: 0.2549 - val_acc: 0.8898
Epoch 179/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2282 - acc: 0.9096 - val_loss: 0.2541 - val_acc: 0.8898
Epoch 180/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2275 - acc: 0.9096 - val_loss: 0.2534 - val_acc: 0.8898
Epoch 181/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2269 - acc: 0.9096 - val_loss: 0.2526 - val_acc: 0.8898
Epoch 182/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2262 - acc: 0.9096 - val_loss: 0.2519 - val_acc: 0.8898
Epoch 183/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2255 - acc: 0.9096 - val_loss: 0.2511 - val_acc: 0.8898
Epoch 184/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.2248 - acc: 0.9096 - val_loss: 0.2504 - val_acc: 0.8898
Epoch 185/5787
3473/

Epoch 237/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1846 - acc: 0.9168 - val_loss: 0.2050 - val_acc: 0.9092
Epoch 238/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1835 - acc: 0.9168 - val_loss: 0.2039 - val_acc: 0.9092
Epoch 239/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1824 - acc: 0.9168 - val_loss: 0.2027 - val_acc: 0.9092
Epoch 240/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1813 - acc: 0.9168 - val_loss: 0.2015 - val_acc: 0.9092
Epoch 241/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1802 - acc: 0.9168 - val_loss: 0.2003 - val_acc: 0.9092
Epoch 242/5787
3473/3473 [==============================] - 0s 3us/sample - loss: 0.1790 - acc: 0.9168 - val_loss: 0.1992 - val_acc: 0.9097
Epoch 243/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 0.1780 - acc: 0.9168 - val_loss: 0.1981 - val_acc: 0.9101
Epoch 244/5787
3473/

...

Epoch 5519/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1475e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5520/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1452e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5521/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1428e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5522/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1404e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5523/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1381e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5524/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1357e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5525/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.1334e-05 - acc: 1.0000 - val_loss: 0.0046 - v

Epoch 5576/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0163e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5577/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0140e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5578/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0118e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5579/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0095e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5580/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0073e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5581/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0050e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5582/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 3.0028e-05 - acc: 1.0000 - val_loss: 0.0046 - v

Epoch 5633/5787
3473/3473 [==============================] - 0s 3us/sample - loss: 2.8910e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5634/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8888e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5635/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8867e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5636/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8846e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5637/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8824e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5638/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8803e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5639/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.8781e-05 - acc: 1.0000 - val_loss: 0.0046 - v

Epoch 5690/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7713e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5691/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7693e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5692/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7673e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5693/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7652e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5694/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7632e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5695/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7611e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5696/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.7591e-05 - acc: 1.0000 - val_loss: 0.0046 - v

Epoch 5747/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6571e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5748/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6551e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5749/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6531e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5750/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6512e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5751/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6492e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5752/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6473e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 0.9978
Epoch 5753/5787
3473/3473 [==============================] - 0s 2us/sample - loss: 2.6453e-05 - acc: 1.0000 - val_loss: 0.0046 - v

In [24]:
# Evaluating the model on the test data
results = model.evaluate(test_data, test_labels)

1929/1929 [==============================] - 0s 12us/sample - loss: 6.8508e-04 - acc: 0.9995


In [25]:
# Saving the model to disk
try:
    model.save("abd_model.h5")
    print("model saved to disk")
except Exception:
    print("something went wrong")

model saved to disk
